# Zotero Cleaner
A quick, hacky script that cleans up a Zotero Group or Library.
Basically just hardcoded fixes to common problems (e.g., Zotero doesn't scrape Lexis or Westlaw, but I want proper case citations in my Zotero Library).

In [2]:
cd ..

/Users/stein/repos/sdk-zotero


In [48]:
import requests
import json
from pyzotero import zotero
from collections import defaultdict
from IPython.display import JSON
import re

In [4]:
with open("/Users/stein/.config/secrets.json") as f:
    secrets = json.load(f)['pyzotero']

In [20]:
TS_paper_zot = zotero.Zotero(4495438, "group", secrets["API_KEY"])

TSZ = TS_paper_zot

In [65]:
items = TSZ.items()

In [66]:
key2item = {item["key"]: item for item in items}

In [67]:
def dicttree(d, level=1):
    for k, v in d.items():
        if isinstance(v, dict):
            print('\t' * level + str(k))
            dicttree(v, level+1)
        else:
            print('\t' * level + str(k) + '\t→ ' + str(v)[:30])


In [68]:
def fix_lexus_case(zotero_item):
    if item['data']['itemType'] in ('attachment', 'case'):
        return
    if not item['data'].get('url', '').startswith('https://plus.lexis.com/document'):
        return
    if item['data'].get('title', ' ').split(":")[0] in ["ARTICLE", "COMMENT"]:
        return
    title_regex=re.compile(r'(?P<casename>.*), (?P<vol>\d+) (?P<reporter>.*) (?P<page>\d+)$')
    title_match = title_regex.match(zotero_item['data']['title'])
    del zotero_item['data']['title']
    zotero_item['data']['itemType'] = 'case'
    zotero_item['data']['caseName'] = title_match.group('casename')
    zotero_item['data']['reporterVolume'] = title_match.group('vol')
    zotero_item['data']['firstPage'] = title_match.group('page')
    zotero_item['data']['reporter'] = title_match.group('reporter')
    TSZ.update_item(zotero_item)

In [69]:
for item in items:
    fix_lexus_case(item)